# Configuration

In [1]:
cd '/Users/xusikun/Desktop/DoubleBazinga/QuantFantasy'

/Users/xusikun/Desktop/DoubleBazinga/QuantFantasy


In [2]:
import os

import pandas as pd 

from datetime import datetime

In [3]:
import backtrader as bt

In [4]:
DATA_BACKTEST_QIML_SAMPLE = 'backtest/data/qiml'

# Data

In [5]:
# historical data
daily_price = pd.read_csv(os.path.join(DATA_BACKTEST_QIML_SAMPLE, 'daily_price.csv'), parse_dates=['datetime'])

# portfolio information
trade_info = pd.read_csv(os.path.join(DATA_BACKTEST_QIML_SAMPLE, 'trade_info.csv'), parse_dates=['trade_date'])

In [6]:
daily_price.head()

,datetime,sec_code,open,high,low,close,volume,openinterest
0,2019-01-02,600466.SH,33.064891,33.496709,31.954503,32.386321,10629352,0
1,2019-01-02,603228.SH,50.660230,51.458513,50.394136,51.120778,426147,0
2,2019-01-02,600315.SH,148.258423,150.480132,148.258423,149.558935,2138556,0
3,2019-01-02,000750.SZ,49.512579,53.154883,48.715825,51.561375,227557612,0
4,2019-01-02,002588.SZ,36.608672,36.608672,35.669988,35.763857,2841517,0


In [7]:
trade_info.head()

,trade_date,sec_code,weight
0,2019-01-31,000006.SZ,0.007282
1,2019-01-31,000008.SZ,0.009783
2,2019-01-31,000025.SZ,0.006928
3,2019-01-31,000090.SZ,0.007234
4,2019-01-31,000536.SZ,0.003536


# First Example: Empty Backtest

In [8]:
# instantiate cerebro
cerebro = bt.Cerebro()

# print starting capital
print("Starting Portfolio Value: {:.2f}".format(cerebro.broker.getvalue()))

# start backtest
cerebro.run()

# print capital after backtest
print("Final Portfolio Value: {:.2f}".format(cerebro.broker.getvalue()))

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00


# A Simple Backtest

## Instantiate Cerebro

In [9]:
# instantiate cerebro
cerebro = bt.Cerebro()

## Import Data

In [10]:
# add data into cerebro
stocks = daily_price['sec_code'].unique()
dates = daily_price.datetime.unique()

for stock in stocks:
    # align the dates
    data = pd.DataFrame(index=dates)
    df = daily_price.query(f"sec_code=='{stock}'")[['open', 'high', 'low', 'close', 'volume', 'openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    
    # handling missing values
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    
    # export data to cerebro
    # 1. create a datafeed object
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime(2019, 1, 2), todate=datetime(2021, 1, 28))
    # 2. put feed into cerebro
    cerebro.adddata(datafeed, name=stock)

## Set Backtest Conditions

In [11]:
# Broker
# initial capital
cerebro.broker.setcash(100000000.0)

# comission
cerebro.broker.setcommission(commission=0.0003)

# slippage
cerebro.broker.set_slippage_perc(perc=0.0001)

In [12]:
# Add analyzers
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # return time series
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # annual return
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # sharpe ratio
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # drawdown

## Trading Strategy

In [13]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        # add portfolio info
        self.buy_stock = trade_info
        
        # read rebalance date
        self.trade_dates = pd.to_datetime(self.buy_stock['trade_date'].unique()).tolist()
        
        # record pst orders, so that we can handle them at rebalance time
        self.order_list = []
        
        # record the last-period holding
        self.buy_stocks_pre = []
        
    def next(self):
        # get the current datetime
        now = self.datas[0].datetime.date(0)
        
        # if it's rebalance date, we do the following 
        if now in self.trade_dates:
            print(f" ---------- {now} is rebalance date ---------- ")
            # 0. before rebalance, we cancel all the unfinished and un-expired orders
            if len(self.order_list) > 0:
                for order in self.order_list:
                    self.cancel(order)
                self.order_list = []
            
            # 1. extract the holdings
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{now}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list)
            
            # 2. In current holdings, sell all stocks that we won't hold anymore
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock)
            if len(sell_stock) > 0:
                print(" ---------- Sell all stocks that we won't hold ---------- ")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0:
                        order = self.close(data=data)
                        self.order_list.append(order)
                        
            # 3. Buy all stocks that we'll hold
            print(" ---------- Buy all stocks we'll hold ---------- ")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0]
                data = self.getdatabyname(stock)
                print(data._name, w, w*0.95)
                order = self.order_target_percent(data=data, target=w*0.95)
                self.order_list.append(order)
            self.buy_stocks_pre = long_list
            
    def notify_order(self, order):
        # un-processed orders
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        # processed orders
        if order.status in [order.Completed, order.Canceled, order.margin]:
            if order.isbuy():
                self.log(f"BUY EXECUTED, ref: {order.ref}, Price: {order.executed.price}, \
                         Cost: {order.executed.value}, Comm: {order.executed.comm}, \
                         Size: {order.executed.size}, Stock: {order.data._name}")
            else:
                self.log(f"SELL EXECUTED, ref: {order.ref}, Price: {order.executed.price}, \
                         Cost: {order.executed.value}, Comm: {order.executed.comm}, \
                         Size: {order.executed.size}, Stock: {order.data._name}")

# add strategy to cerebro
cerebro.addstrategy(TestStrategy)

0

## Get Backtest Results

In [14]:
# start backtest
result = cerebro.run()

 ---------- 2019-01-31 is rebalance date ---------- 
long_list ['000006.SZ', '000008.SZ', '000025.SZ', '000090.SZ', '000536.SZ', '000587.SZ', '000598.SZ', '000612.SZ', '000636.SZ', '000656.SZ', '000690.SZ', '000712.SZ', '000766.SZ', '000807.SZ', '000829.SZ', '000877.SZ', '000980.SZ', '000999.SZ', '002002.SZ', '002048.SZ', '002051.SZ', '002074.SZ', '002110.SZ', '002127.SZ', '002128.SZ', '002131.SZ', '002152.SZ', '002195.SZ', '002308.SZ', '002358.SZ', '002359.SZ', '002375.SZ', '002400.SZ', '002408.SZ', '002437.SZ', '002463.SZ', '002465.SZ', '002642.SZ', '002707.SZ', '002745.SZ', '002818.SZ', '300001.SZ', '300010.SZ', '300058.SZ', '300113.SZ', '300146.SZ', '300166.SZ', '300266.SZ', '300376.SZ', '300450.SZ', '600006.SH', '600039.SH', '600053.SH', '600056.SH', '600062.SH', '600141.SH', '600151.SH', '600158.SH', '600169.SH', '600259.SH', '600260.SH', '600280.SH', '600366.SH', '600373.SH', '600392.SH', '600393.SH', '600428.SH', '600478.SH', '600500.SH', '600525.SH', '600528.SH', '600582.SH', 

ValueError: cannot convert float NaN to integer

In [ ]:
# extract backtest results
strat = result[0]

In [ ]:
# return daily return sequence
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())

In [ ]:
# print
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())